# Aufgabe-G-WiederholteFragen

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/G_fragen herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen ml_2021 anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Die Datensammlung enthält Fragen aus einem Forum. Es sind Paare aus Fragen gleichen Inhalts geboten.

Erstellen Sie ein tiefes Neuronales Netzwerk, dass dem Nutzer schon bei der Eingabe einer neuen Frage gleich ähnliche Fragen anbietet. 
* Je mehr Text der Benutzer eingibt, desto präziser sollen die ähnlichen Fragen sein! 




# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

In [5]:
# Fügen Sie ihren Code hier ein!

Vorgehen:
- Finden von unnötigen Wörtern (in, wie usw.)
- Extrahieren der "wertvollen" Informationswerten
- Einteilen der Fragen in Kategorien und Lernen KI anhand der gelieferten Paare
- Überprüfung anhand der Testfragen
- Erstellung einer Eingabemaske, mit Testfragen zur überprüfung der KI

# Einlesen der Daten

## Einlesen der Trainingsdaten

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

train_data = pd.read_csv(
    "./fragen/train.csv",
    index_col="id"
)
train_data.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## Einlesen der Testdaten

In [8]:
test_data = pd.read_csv(
    "./fragen/test.csv",
    index_col="test_id"
)

test_data.head()

/tmp/ipykernel_227555/3543877039.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(


,question1,question2
test_id,,
0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,What but is the best way to send money from Ch...,What you send money to China?
3,Which food not emulsifiers?,What foods fibre?
4,"How ""aberystwyth"" start reading?",How their can I start reading?
